# Week 3 Lab

Shrihan Avilala

## Prep Work (10 Points)




$I  = \int_a^b (20 - x^2)\,dx \\[6pt] $
  $ = \int_a^b 20\,dx - \int_a^b x^2\,dx \\[6pt]$
   $= \left[ 20x - \tfrac{x^3}{3} \right]_a^b  \\[6pt]$
  $ = \big(20b - 20a\big) - \left(\tfrac{b^3}{3} - \tfrac{a^3}{3}\right) \\[6pt]$
   $ = 20(b-a) - \tfrac{1}{3}\big(b^3 - a^3\big) $
 

In [2]:
def f(x:float) -> float:
    return 20-x*x

def integral_true(a:float, b:float) -> float:
    return 20*(b-a) - (b**3-a**3)/3

## 2. Basic Monte Carlo (25 points)

$\mathbb{E^p}[I_n^{\text{Basic}}] = \int_{a}^{b} f(x)\,dx = I $ 

In [3]:
from typing import Callable
import numpy
def mc_basic(f: Callable[[float], float], a: float, b: float, n: int) -> tuple[float, float]:
    estimates = numpy.empty(n)
    for index in range(n-1):
        f_x_rand = f(numpy.random.uniform(a, b))
        probability = 1/(b-a)
        estimates[index] = f_x_rand/probability
    I_n = numpy.mean(estimates)
    basic_var  = numpy.var(estimates)/n
    return I_n, basic_var
        

$ \displaystyle Var(I_n^{\text{Basic}}) = \frac{1}{n} Var(\frac{f(x)}{p(x)})\\[6pt]$
$ \displaystyle= \frac{1}{n}\,Var(\,(20-x^2)(b-a)\,) \\[6pt]$
$ \displaystyle= \frac{1}{n}\,(b-a)\,Var(\,20-x^2\,) \\[6pt]$
$ \displaystyle= \frac{1}{n}\,(b-a)(\,\mathbb{E[\,(20-x^2)^2\,]}-\mathbb{E[\,(20-x^2)\,]}^2) \\[6pt]$
$ \displaystyle= \frac{1}{n}\,(b-a)(\,\mathbb{E[\,400-40x^2+x^4\,]} - \mathbb{E[\,(20-x^2)\,]}^2\,) \\[6pt]$
$ \displaystyle= \frac{1}{n}\,(b-a)(\frac{1}{b-a}\int_a^b 400-40x^2+x^4\,dx - (\,\frac{1}{b-a}\int_a^b 20-x^2\,dx\,)^2) \\[6pt]$
$ \displaystyle= \frac{1}{n}\,(b-a)(400 - \frac{40}{3}\,(a^2 + ab + b^2) $
$ + \frac{1}{5}\,(a^4 + a^3b + a^2b^2 + ab^3 + b^4) - \left(20 - \tfrac{1}{3}(a^2 + ab + b^2)\right)^2)\\[6pt] $
$ \displaystyle= \frac{1}{n}\,(b-a)\left(\frac{4}{45}(a^4+b^4) - \frac{1}{45}(a^3b+ab^3) - \frac{2}{15}a^2b^2\right) $


In [4]:
def mc_basic_variance_true(a: float, b: float, n: int) -> float:
    return (1/n)*(b-a)*( (4*(a**4+b**4)/45) - ((b*a**3+a*b**3)/45) - (2*a**2*b**2)/15 )

In [31]:
n=100000
a=-1
b=3

integral_true_metric = integral_true(a, b)
trial = mc_basic(f, a, b, n)
integral_estimate = trial[0]
difference = integral_estimate-integral_true_metric
var_estimate = trial[1]
var_estimate_comparable=var_estimate*n
var_true = mc_basic_variance_true(a, b, n)
var_true_comparable = var_true*n

integral_true_metric, integral_estimate, difference, var_estimate, var_estimate_comparable, var_true, var_estimate_comparable

(70.66666666666667,
 70.69985113575589,
 0.033184469089221125,
 0.001074439887620132,
 107.4439887620132,
 0.00027022222222222226,
 107.4439887620132)

## 3. Antithetic uniform samples (15 points)

$\displaystyle \mathbb{E^p}[I_n^{\text{Antithetic}}] = \mathbb{E^p} [\, \frac{1}{n/2} \sum_{i=1}^{n/2} \frac{1}{2} 
\left( \frac{f(x_i)}{p(x_i)} + \frac{f(\hat{x}_i)}{p(\hat{x}_i)} \right)\,]$ 

$ \displaystyle = \frac{1}{n} \, \mathbb{E^p} [\, \sum_{i=1}^{n/2}
\left( \frac{f(x_i)}{p(x_i)} + \frac{f(\hat{x}_i)}{p(\hat{x}_i)} \right)\,] $

$ \displaystyle = \frac{b-a}{n} \, \mathbb{E^p} [\, \sum_{i=1}^{n/2}
\left( f(x_i) + f(a+b-x_i) \right)\,] \qquad \text{(Probability is Uniform and antithetic pairs are symmetric)}$

$ \displaystyle = \frac{b-a}{n} \, \sum_{i=1}^{n/2}
\mathbb{E^p} [\left( f(x_i) + f(a+b-x_i) \right)]\,$

$ \displaystyle = \frac{b-a}{2} \,
\mathbb{E^p} [\left( f(x_i) + f(a+b-x_i) \right)]\, $

$ \displaystyle = \frac{1}{2} \,
\int_a^b \left( f(x) + f(a+b-x) \right)dx\, $

$ \displaystyle = \frac{1}{2} \,
\int_a^b \left( 20-x^2+20-(a+b-x)^2 \right)dx\, $

$ 
= 20(b-a) - \frac{1}{3}\big(b^3 - a^3\big) $

$ \displaystyle =I$

In [7]:
def mc_antithetic(f: Callable[[float], float], a: float, b: float, n: int) -> tuple[float, float]:
    estimates = numpy.empty(n//2, dtype=numpy.float64)
    ant_estimates = numpy.empty(n//2, dtype=numpy.float64)
    for index in range(n//2-1):
        x_rand = numpy.random.uniform(a, b)
        estimates[index] = f(x_rand)*(b-a)
        ant_estimates[index] = f(a+b-x_rand)*(b-a)
    I_n = (numpy.mean(estimates)+numpy.mean(ant_estimates))/2
    antithetic_var = numpy.var( numpy.concatenate([estimates,ant_estimates]), ddof=1)/n
    return I_n, antithetic_var

In [8]:
n=100000
a=-1
b=3

integral_true_metric = integral_true(a, b)
trial = mc_antithetic(f, a, b, n)
integral_estimate = trial[0]
difference = integral_estimate-integral_true_metric
var_estimate = trial[1]
var_estimate_comparable=var_estimate*n
integral_true_metric, integral_estimate, difference, var_estimate, var_estimate_comparable

70.66666666666667 70.65628217833321 0.0010847212993431927


The variance is smaller with antithetic sampling compared to basic sampling. This is expected because random draws that are outliers tend to be cancelled out by picking a number that is symmetrically on the opposite end of the interval. Consequently, the average tends to remains stable, which helps reduce the variance.

## 4. Control Variate (25 points)

$ \displaystyle \text{Let} \int_a^b {h(x)p(x)}\,dx = H. $

$ \displaystyle \mathbb{E^p[I_n^\text{ControlVariate}]} = \mathbb{E^p} [\, \frac{1}{n} \sum_{i=1}^{n}
 \frac{f(x_i)-h(x_i)}{p(x_i)} + H]\, $

$ \displaystyle = \frac{1}{n} \sum_{i=1}^{n}\mathbb{E^p[\frac{f(x_i)}{p(x_i)}]} - \frac{1}{n}\sum_{i=1}^{n} \mathbb{E^p[\frac{h(x_i)}{p(x_i)}]}+\mathbb{E^p[H]}$

$ \displaystyle = \frac{1}{n} \sum_{i=1}^{n}\mathbb{E^p[\frac{f(x_i)}{p(x_i)}]} + \int_a^b {h(x)p(x)}\,dx - \int_a^b {h(x)p(x)}\,dx$

$ = I $

In [9]:
import math
def h1(x: float) -> float:
    return 20 * math.cos(x/3)

def h2(x: float) -> float:
    return 20*math.sin(x/3)

def H1(a: float, b: float) -> float:
    # Since int(cos(x/3)) = 3sin(x/3)
    return -60*math.sin(a/3) + 60*math.sin(b/3)

def H2(a: float, b: float) -> float:
    return 60*math.cos(a/3) - 60*math.cos(b/3)

In [10]:

def mc_control_variate(f: Callable[[float], float], a: float, b: float, n: int, h: Callable[[float], float], H: Callable[[float, float], float]) -> tuple[float, float]:
    estimates = numpy.empty(n)
    for each in range(n-1):
        x_rand = numpy.random.uniform(a, b)
        estimates[each] = (f(x_rand) - h(x_rand))*(b-a)
        
    I_n = numpy.mean(estimates) + H(a, b)
    control_var = numpy.var(estimates) / n
    return I_n, control_var

In [11]:
n=100000
a=-1
b=3

integral_true_metric = integral_true(a, b)
trial = mc_control_variate(f, a, b, n, h1, H1)
integral_estimate = trial[0]
difference = integral_estimate-integral_true_metric
var_estimate = trial[1]
var_estimate_comparable=var_estimate*n

integral_true_metric, integral_estimate, difference, var_estimate, var_estimate_comparable

trial = mc_control_variate(f, a, b, n, h2, H2)
integral_estimate = trial[0]
difference = integral_estimate-integral_true_metric
var_estimate = trial[1]
var_estimate_comparable=var_estimate*n

integral_true_metric, integral_estimate, difference, var_estimate, var_estimate_comparable

0.12618400862083945

$20\cos(x/3)$ has less variance because it has a higher covariance with f(x), which means that the estimations, which are the difference of f(x) and h(x), are brought closer to 0, which inhernetly reduces variance.

The variance was less than the basic Monte Carlo estimator, which was what I expected.

## 5. Importance Sampling (25 Points)

Let $ \tilde{p}(\cdot)$ be the uniform distribution and $p(\cdot)$ be the normal distribution. 

$ \displaystyle \mathbb{E^p[I_n^\text{Importance}]} = \mathbb{E^p} [\, \frac{1}{n} \sum_{i=1}^{n}
 \frac{f(x_i)\,\tilde{p}(x_i)}{p(x_i)}]\, $

$ \displaystyle  = \int_a^b { \frac{f(x)\,\tilde{p}(x)}{p(x)}p(x)\,dx} $

$ \displaystyle  = \int_a^b {f(x)\,\tilde{p}(x)\,dx} $

$ =  \displaystyle \mathbb{E^{\tilde{p}}[I_n^\text{Importance}]} $

$ = I $

In [24]:
from scipy.stats import norm

def mc_importance(f: Callable[[float], float], a: float, b: float, n: int, sigma: float) -> tuple[float, float, float]:
        
        estimates = numpy.zeros(n)
        for each in range(n-1):
                # f(x) = 20-x^2 is centered at x=0
                x_rand = numpy.random.normal(0, sigma)
                if x_rand < - 1 or x_rand>3:
                        continue
                estimates[each] = ( f(x_rand) * (1/(b-a))) / (norm.pdf(x_rand, 0, sigma))
        
        estimates = estimates[estimates != 0]
        I_n = numpy.mean(estimates) 
        control_var = numpy.var(estimates) / len(estimates)
        return I_n, control_var, len(estimates)
                
        

In [25]:
n=100000
a=-1
b=3

integral_true_metric = integral_true(a, b)
for each in range(1, 11):
    trial = mc_importance(f, a, b, n, each)
    integral_estimate = trial[0]
    difference = integral_estimate-integral_true_metric
    var_estimate = trial[1]
    var_estimate_comparable=var_estimate*n
    n_used = trial[2]
    print(integral_true_metric, integral_estimate, difference, var_estimate, var_estimate_comparable, n_used)

70.66666666666667 21.167101570652267 -49.49956509601441 0.010613028771691084 1061.3028771691083 83946
70.66666666666667 28.300190018285125 -42.36647664838155 0.00029199908457026185 29.199908457026186 62611
70.66666666666667 37.438193176253755 -33.22847349041292 9.586548972390413e-06 0.9586548972390413 46986
70.66666666666667 47.518946297613375 -23.147720369053296 0.00031322051389043787 31.322051389043786 37341
70.66666666666667 57.95855542979929 -12.708111236867381 0.0010767261215863459 107.67261215863459 30453
70.66666666666667 68.55127274141353 -2.1153939252531444 0.0024023894018727586 240.23894018727586 25701
70.66666666666667 79.16511281828453 8.498446151617856 0.004382970906691908 438.2970906691908 22126
70.66666666666667 90.38479338818061 19.71812672151394 0.006688533104636039 668.853310463604 19656
70.66666666666667 101.05226857986135 30.385601913194677 0.010215057913159862 1021.5057913159861 17487


Standard deviation $\sigma=6$ was the best approximation because it had the most covariance with $f(x)$

The variance was roughly 5-30x less with importance sampling than basic sampling, which is expected because importance sampling reduces variance by cancelling out much of the highs and lows of the function by dividing them with corresponding probabilities.

As $\sigma$ increased, n_used decreased because as the normal distribution gets wider, values further away from the mean are more likely to be drawn. Since we limit draws to the interval $[-1, 3]$, this means more values are not selected.